# SBERT

In [3]:
# coding: utf-8
import json
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.semi_supervised import LabelSpreading
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, f1_score
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import bert_score
import torch
random_seed=42

In [4]:
def calibrate_probability(p, p_value):
    if p == p_value:
        return 0.5
    elif p > p_value:
        return 0.5 + (p - p_value) * (1 - 0.5) / (1 - p_value)
    else:  # p < p_value
        return 0 + (p - 0) * (0.5 - 0) / (p_value - 0)

In [14]:
test_data_aware_all = pd.read_json('/content/drive/MyDrive/HIT/SHROOM/data/test.model-aware.json')
df_all1 = pd.DataFrame()

# Before this, calculate the threshold and accuracy of the model in the validation set

for task in ['DM','MT','PG']: # aware
  if task == 'DM':
    p_value1 = 0.26 # 0.26
    p_value2 = 0.28 # 0.28
    p_value3 = 0.26 #0.28
    w1 = 0.79 #0.75
    w2 = 0.76 #0.76
    w3 = 0.76 #0.75
  elif task == 'MT':
    p_value1 = 0.12 # 0.16
    p_value2 = 0.14 #0.2
    p_value3 = 0.14 #0.18
    w1 = 0.78 #0.86
    w2 = 0.78 #0.88
    w3 = 0.77 #0.87
  elif task == 'PG':
    p_value1 = 0.16 #0.24
    p_value2 = 0.22 #0.22
    p_value3 = 0.22 #0.2
    w1 = 0.82 #0.7
    w2 = 0.8 #0.69
    w3 = 0.8 #0.69
  df=test_data_aware_all.copy()
  df=df[df['task']==task]
  df.index=np.arange(df.shape[0])

  # DM:tgt   MT:tgt   PG:src
  df.loc[df['task'].isin(['DM', 'MT']), 'tgt'] = df['tgt']
  df.loc[df['task']=='PG', 'tgt'] = df['src']

  model = SentenceTransformer('WhereIsAI/UAE-Large-V1')
  df['text1_vector'] = df['tgt'].apply(lambda x: model.encode(x))
  df['text2_vector'] = df['hyp'].apply(lambda x: model.encode(x))
  df['cosine_similarity'] = df.apply(lambda row: cosine_similarity([row['text1_vector']], [row['text2_vector']])[0][0], axis=1)
  df['cosine_similarity'] = 1 - df['cosine_similarity']
  df['pred1'] = df['cosine_similarity'].apply(lambda x: calibrate_probability(x,p_value1))

  model = SentenceTransformer('llmrails/ember-v1')
  df['text1_vector'] = df['tgt'].apply(lambda x: model.encode(x))
  df['text2_vector'] = df['hyp'].apply(lambda x: model.encode(x))
  df['cosine_similarity'] = df.apply(lambda row: cosine_similarity([row['text1_vector']], [row['text2_vector']])[0][0], axis=1)
  df['cosine_similarity'] = 1 - df['cosine_similarity']
  df['pred2'] = df['cosine_similarity'].apply(lambda x: calibrate_probability(x,p_value2))

  model = SentenceTransformer('BAAI/bge-large-en-v1.5')
  df['text1_vector'] = df['tgt'].apply(lambda x: model.encode(x))
  df['text2_vector'] = df['hyp'].apply(lambda x: model.encode(x))
  df['cosine_similarity'] = df.apply(lambda row: cosine_similarity([row['text1_vector']], [row['text2_vector']])[0][0], axis=1)
  df['cosine_similarity'] = 1 - df['cosine_similarity']
  df['pred3'] = df['cosine_similarity'].apply(lambda x: calibrate_probability(x,p_value3))

  df['pred'] = df['pred1']*(w1/(w1+w2+w3)) + df['pred2']*(w2/(w1+w2+w3)) + df['pred3']*(w3/(w1+w2+w3))




  df_all1 = pd.concat([df_all1, df])
df_all1 = df_all1.sort_values('id')

In [19]:
df_all1.index=np.arange(df_all1.shape[0])

In [20]:
# model-aware track prediction result output
df_all1['p(Hallucination)'] = df_all1['pred']
df_all1['label'] = (df_all1['pred']>=0.5).astype(int).tolist()

test_data_aware_all = df_all1

path_val_model_aware_output = "test.model-aware.json"
output_json = []
for i in np.arange(test_data_aware_all.shape[0]):
    output_label = 'Hallucination' if test_data_aware_all.loc[i,'label'] == 1 else 'Not Hallucination'
    prob=test_data_aware_all.loc[i,'p(Hallucination)']
    id=test_data_aware_all.loc[i,'id']
    item_to_json = {"label":output_label, "p(Hallucination)":np.float64(prob), "id":int(id)}
    output_json.append(item_to_json)
f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()

In [23]:
test_data_aware_all = pd.read_json('/content/drive/MyDrive/HIT/SHROOM/data/test.model-agnostic.json')
df_all2 = pd.DataFrame()

# Before this, calculate the threshold and accuracy of the model in the validation set

for task in ['DM','MT','PG']: # agnostic
  if task == 'DM':
    p_value1 = 0.26
    p_value2 = 0.28
    p_value3 = 0.28
    w1 = 0.75
    w2 = 0.76
    w3 = 0.75
  elif task == 'MT':
    p_value1 = 0.16
    p_value2 = 0.2
    p_value3 = 0.18
    w1 = 0.86
    w2 = 0.88
    w3 = 0.87
  elif task == 'PG':
    p_value1 = 0.24
    p_value2 = 0.22
    p_value3 = 0.2
    w1 = 0.7
    w2 = 0.69
    w3 = 0.69
  df=test_data_aware_all.copy()
  df=df[df['task']==task]
  df.index=np.arange(df.shape[0])

  # DM:tgt   MT:tgt   PG:src
  df.loc[df['task'].isin(['DM', 'MT']), 'tgt'] = df['tgt']
  df.loc[df['task']=='PG', 'tgt'] = df['src']

  model = SentenceTransformer('WhereIsAI/UAE-Large-V1')
  df['text1_vector'] = df['tgt'].apply(lambda x: model.encode(x))
  df['text2_vector'] = df['hyp'].apply(lambda x: model.encode(x))
  df['cosine_similarity'] = df.apply(lambda row: cosine_similarity([row['text1_vector']], [row['text2_vector']])[0][0], axis=1)
  df['cosine_similarity'] = 1 - df['cosine_similarity']
  df['pred1'] = df['cosine_similarity'].apply(lambda x: calibrate_probability(x,p_value1))

  model = SentenceTransformer('llmrails/ember-v1')
  df['text1_vector'] = df['tgt'].apply(lambda x: model.encode(x))
  df['text2_vector'] = df['hyp'].apply(lambda x: model.encode(x))
  df['cosine_similarity'] = df.apply(lambda row: cosine_similarity([row['text1_vector']], [row['text2_vector']])[0][0], axis=1)
  df['cosine_similarity'] = 1 - df['cosine_similarity']
  df['pred2'] = df['cosine_similarity'].apply(lambda x: calibrate_probability(x,p_value2))

  model = SentenceTransformer('BAAI/bge-large-en-v1.5')
  df['text1_vector'] = df['tgt'].apply(lambda x: model.encode(x))
  df['text2_vector'] = df['hyp'].apply(lambda x: model.encode(x))
  df['cosine_similarity'] = df.apply(lambda row: cosine_similarity([row['text1_vector']], [row['text2_vector']])[0][0], axis=1)
  df['cosine_similarity'] = 1 - df['cosine_similarity']
  df['pred3'] = df['cosine_similarity'].apply(lambda x: calibrate_probability(x,p_value3))

  df['pred'] = df['pred1']*(w1/(w1+w2+w3)) + df['pred2']*(w2/(w1+w2+w3)) + df['pred3']*(w3/(w1+w2+w3))




  df_all2 = pd.concat([df_all2, df])
df_all2 = df_all2.sort_values('id')

df_all2.index=np.arange(df_all2.shape[0])

In [24]:
# model-agnostic track prediction result output
df_all2['p(Hallucination)'] = df_all2['pred']
df_all2['label'] = (df_all2['pred']>=0.5).astype(int).tolist()

test_data_agnostic_all = df_all2

path_val_model_aware_output = "test.model-agnostic.json"
output_json = []
for i in np.arange(test_data_agnostic_all.shape[0]):
    output_label = 'Hallucination' if test_data_agnostic_all.loc[i,'label'] == 1 else 'Not Hallucination'
    prob=test_data_agnostic_all.loc[i,'p(Hallucination)']
    id=test_data_agnostic_all.loc[i,'id']
    item_to_json = {"label":output_label, "p(Hallucination)":np.float64(prob), "id":int(id)}
    output_json.append(item_to_json)
f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()

UniEval

In [ ]:
!git clone https://github.com/maszhongming/UniEval.git
import os
os.chdir('/content/UniEval')
!pip install -r requirements.txt
import nltk
nltk.download('punkt')
from utils import convert_to_json
from metric.evaluator import get_evaluator
def UniEval(output_list,src_list,task='fact'):
  data = convert_to_json(output_list=output_list, src_list=src_list)
  evaluator = get_evaluator(task)
  eval_scores = evaluator.evaluate(data, print_result=True)
  return eval_scores

In [ ]:
for task in ['DM','MT','PG']:
  df=pd.read_json(r'/content/drive/MyDrive/HIT/SHROOM/data/val.model-aware.v2.json')
  df=df[df['task']==task]
  df.index=np.arange(df.shape[0])
  # df=pd.read_json(r'/content/drive/MyDrive/HIT/SHROOM/data/test.model-aware.json')
  # df['label']=-1
  # df['p(Hallucination)']=-1


  # DM:tgt   MT:tgt   PG:src
  df.loc[df['task'].isin(['DM', 'MT']), 'tgt'] = df['tgt']
  df.loc[df['task']=='PG', 'tgt'] = df['src']
  df['label'].replace('Hallucination',1,inplace=True)
  df['label'].replace('Not Hallucination',0,inplace=True)


  x=[]
  for i in np.arange(0,df.shape[0],10):
    a=UniEval(df.loc[i:i+10-1,'hyp'].values, df.loc[i:i+10-1,'tgt'].values)
    for i in a:
      x.append(i['consistency'])
  df['UniEval'] = x
  df['UniEval'] = 1 - df['UniEval']

  # output
  for zhiding in np.arange(0.1,1,0.02):
    df['label_pre'] = df['UniEval'].apply(lambda x: 1 if x>zhiding else 0)
    print('阈值：',round(zhiding,2),' 准确率：',round(accuracy_score(df['label'], df['label_pre']),2))
  print('Spearman：', round(spearmanr(df['UniEval'], df['p(Hallucination)'])[0], 2))

In [ ]:
for task in ['DM','MT','PG']:
  df=pd.read_json(r'/content/drive/MyDrive/HIT/SHROOM/data/val.model-agnostic.json')
  df=df[df['task']==task]
  df.index=np.arange(df.shape[0])
  # df=pd.read_json(r'/content/drive/MyDrive/HIT/SHROOM/data/test.model-aware.json')
  # df['label']=-1
  # df['p(Hallucination)']=-1


  # DM:tgt   MT:tgt   PG:src
  df.loc[df['task'].isin(['DM', 'MT']), 'tgt'] = df['tgt']
  df.loc[df['task']=='PG', 'tgt'] = df['src']
  df['label'].replace('Hallucination',1,inplace=True)
  df['label'].replace('Not Hallucination',0,inplace=True)

  x=[]
  for i in np.arange(0,df.shape[0],10):
    a=UniEval(df.loc[i:i+10-1,'hyp'].values, df.loc[i:i+10-1,'tgt'].values)
    for i in a:
      x.append(i['consistency'])

  df['UniEval'] = x
  df['UniEval'] = 1 - df['UniEval']


  # ouput
  for zhiding in np.arange(0.1,1,0.02):
    df['label_pre'] = df['UniEval'].apply(lambda x: 1 if x>zhiding else 0)
    print('阈值：',round(zhiding,2),' 准确率：',round(accuracy_score(df['label'], df['label_pre']),2))
  print('Spearman：', round(spearmanr(df['UniEval'], df['p(Hallucination)'])[0], 2))

In [ ]:
# model-aware track prediction result output

test_data_aware_all = pd.read_json('/content/drive/MyDrive/HIT/SHROOM/data/test.model-aware.json')
df_all1 = pd.DataFrame()

for task in ['DM','MT','PG']:
  if task == 'DM':
    p_value1 = 0.14
    w1 = 0.74
  elif task == 'MT':
    p_value1 = 0.16
    w1 = 0.73
  elif task == 'PG':
    p_value1 = 0.22
    w1 = 0.83

  df=test_data_aware_all.copy()
  df=df[df['task']==task]
  df.index=np.arange(df.shape[0])

  # DM:tgt   MT:tgt   PG:src
  df.loc[df['task'].isin(['DM', 'MT']), 'tgt'] = df['tgt']
  df.loc[df['task']=='PG', 'tgt'] = df['src']


  x=[]
  for i in np.arange(0,df.shape[0],10):
    a=UniEval(df.loc[i:i+10-1,'hyp'].values, df.loc[i:i+10-1,'tgt'].values)
    for i in a:
      x.append(i['consistency'])
  df['UniEval'] = x
  df['UniEval'] = 1 - df['UniEval']

  df['pred'] = df['UniEval']


  df_all1 = pd.concat([df_all1, df])
df_all1 = df_all1.sort_values('id')
df_all1.index=np.arange(df_all1.shape[0])

# Test set 2 output results
df_all1['p(Hallucination)'] = df_all1['pred']
df_all1['label'] = (df_all1['pred']>=0.5).astype(int).tolist()

test_data_aware_all = df_all1

path_val_model_aware_output = "test.model-aware.json"
output_json = []
for i in np.arange(test_data_aware_all.shape[0]):
    output_label = 'Hallucination' if test_data_aware_all.loc[i,'label'] == 1 else 'Not Hallucination'
    prob=test_data_aware_all.loc[i,'p(Hallucination)']
    id=test_data_aware_all.loc[i,'id']
    item_to_json = {"label":output_label, "p(Hallucination)":np.float64(prob), "id":int(id)}
    output_json.append(item_to_json)
f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()

In [ ]:
# model-agnostic track prediction result output

test_data_aware_all = pd.read_json('/content/drive/MyDrive/HIT/SHROOM/data/test.model-agnostic.json')
df_all2 = pd.DataFrame()

for task in ['DM','MT','PG']:
  if task == 'DM':
    p_value1 = 0.16
    w1 = 0.7
  elif task == 'MT':
    p_value1 = 0.5
    w1 = 0.8
  elif task == 'PG':
    p_value1 = 0.42
    w1 = 0.72
  df=test_data_aware_all.copy()
  df=df[df['task']==task]
  df.index=np.arange(df.shape[0])

  # DM:tgt   MT:tgt   PG:src
  df.loc[df['task'].isin(['DM', 'MT']), 'tgt'] = df['tgt']
  df.loc[df['task']=='PG', 'tgt'] = df['src']

  x=[]
  for i in np.arange(0,df.shape[0],10):
    a=UniEval(df.loc[i:i+10-1,'hyp'].values, df.loc[i:i+10-1,'tgt'].values)
    for i in a:
      x.append(i['consistency'])
  df['UniEval'] = x
  df['UniEval'] = 1 - df['UniEval']


  df['pred'] = df['UniEval']




  df_all2 = pd.concat([df_all2, df])
df_all2 = df_all2.sort_values('id')
df_all2.index=np.arange(df_all2.shape[0])

# Test set 1 output results
df_all2['p(Hallucination)'] = df_all2['pred']
df_all2['label'] = (df_all2['pred']>=0.5).astype(int).tolist()

test_data_agnostic_all = df_all2

path_val_model_aware_output = "test.model-agnostic.json"
output_json = []
for i in np.arange(test_data_agnostic_all.shape[0]):
    output_label = 'Hallucination' if test_data_agnostic_all.loc[i,'label'] == 1 else 'Not Hallucination'
    prob=test_data_agnostic_all.loc[i,'p(Hallucination)']
    id=test_data_agnostic_all.loc[i,'id']
    item_to_json = {"label":output_label, "p(Hallucination)":np.float64(prob), "id":int(id)}
    output_json.append(item_to_json)
f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()